In [ ]:
import tensorflow as tf


conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, (3,3), activation="tanh", input_shape=(64, 64, 3), kernel_initializer="random_uniform", kernel_regularizer=tf.keras.regularizers.l2(l=0.01)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(5, (5,5), activation="tanh", kernel_regularizer=tf.keras.regularizers.l2(l=0.01)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(20, activation='relu', kernel_initializer="random_uniform"),
    tf.keras.layers.Dense(1, activation='relu', kernel_initializer="random_uniform")
],
name="TesteConv"
)

tf.keras.utils.set_random_seed(123)

mse = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
opt = tf.keras.optimizers.SGD(learning_rate=0.1)
conv_model.compile(
    "adam",
    loss="mean_squared_error",
    metrics=['accuracy']
)
conv_model.build(conv_x_train.shape, )
# conv_model.summary()

results = conv_model.fit(
    conv_x_train,
    conv_y_train,
    validation_data=(conv_x_test, y_test),
    epochs=50,
)

conv_predictions = np.round(conv_model.predict(conv_x_test).flatten())
conv_correct_predictions = sum(conv_predictions == y_test)
conv_accuracy = 100 * conv_correct_predictions / len(conv_predictions)
conv_false_positive = sum(conv_predictions > y_test)
conv_false_negative = sum(conv_predictions < y_test)

plt.plot(results.history['accuracy'], label='accuracy')
plt.plot(results.history['val_accuracy'], label='val_accuracy')
plt.ylim([0, 1.1])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')